In [26]:
# from importlib import reload
#
# from keras.src.metrics.accuracy_metrics import accuracy
# from seqeval.metrics import accuracy_score
#
# import src.preprocessing_pipeline
#
# reload(src.preprocessing_pipeline)
#
# from src.preprocessing_pipeline import DataPreprocessor
#
# preprocessor = DataPreprocessor()
# processed_df = preprocessor.run_pipeline()
# processed_df.head()

Data saved successfully to ../data/2023_stock_with_features.csv


,date,ticker,open,close,min,max,avg,quantity,volume,ibovespa_close,day_of_week,daily_return,price_range,volume_per_quantity,target,rolling_close_5,rolling_std_5,rolling_return_5,rolling_volume_5,momentum_5
25,2023-01-09,ABEV3,14.29,14.40,14.27,14.47,14.39,24448600,3.519505e+08,109227.0,1,0.007698,0.20,14.395529,14.54,14.244,0.063875,-0.002636,3.625215e+08,0.010952
26,2023-01-09,VALE3,91.53,92.44,91.25,92.99,92.43,19285800,1.782778e+09,109227.0,1,0.009942,1.74,92.439943,93.58,90.256,1.346804,0.007654,2.256946e+09,0.024198
27,2023-01-09,PETR4,23.50,23.87,23.25,24.00,23.77,46385200,1.102685e+09,109227.0,1,0.015745,0.75,23.772347,24.09,23.186,0.631174,0.002385,1.986397e+09,0.029501
28,2023-01-09,BBDC4,14.31,14.56,14.21,14.74,14.53,73674800,1.070745e+09,109227.0,1,0.017470,0.53,14.533399,15.10,14.500,0.463735,-0.000755,1.595722e+09,0.004138
29,2023-01-09,ITUB4,25.00,25.41,24.89,25.66,25.35,25669400,6.508859e+08,109227.0,1,0.016400,0.77,25.356492,25.90,24.510,0.523307,0.004459,7.684781e+08,0.036720


In [27]:
import pandas as pd

df = pd.read_csv("../data/2023_stock_with_features.csv")
df

,date,ticker,open,close,min,max,avg,quantity,volume,ibovespa_close,day_of_week,daily_return,price_range,volume_per_quantity,target,rolling_close_5,rolling_std_5,rolling_return_5,rolling_volume_5,momentum_5
0,2023-01-09,ABEV3,14.29,14.40,14.27,14.47,14.39,24448600,3.519505e+08,109227.0,1,0.007698,0.20,14.395529,14.54,14.244,0.063875,-0.002636,3.625215e+08,0.010952
1,2023-01-09,VALE3,91.53,92.44,91.25,92.99,92.43,19285800,1.782778e+09,109227.0,1,0.009942,1.74,92.439943,93.58,90.256,1.346804,0.007654,2.256946e+09,0.024198
2,2023-01-09,PETR4,23.50,23.87,23.25,24.00,23.77,46385200,1.102685e+09,109227.0,1,0.015745,0.75,23.772347,24.09,23.186,0.631174,0.002385,1.986397e+09,0.029501
3,2023-01-09,BBDC4,14.31,14.56,14.21,14.74,14.53,73674800,1.070745e+09,109227.0,1,0.017470,0.53,14.533399,15.10,14.500,0.463735,-0.000755,1.595722e+09,0.004138
4,2023-01-09,ITUB4,25.00,25.41,24.89,25.66,25.35,25669400,6.508859e+08,109227.0,1,0.016400,0.77,25.356492,25.90,24.510,0.523307,0.004459,7.684781e+08,0.036720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,2023-11-16,ABEV3,13.70,13.67,13.55,13.80,13.66,45772900,6.256602e+08,124576.0,4,-0.002190,0.25,13.668791,13.60,13.486,0.137949,-0.000720,2.831401e+08,0.013644
1066,2023-11-16,PETR4,35.98,35.55,35.50,36.49,35.81,72361200,2.591860e+09,124576.0,4,-0.011951,0.99,35.818366,36.71,35.128,0.801293,0.008634,1.756716e+09,0.012013
1067,2023-11-16,ITUB4,30.00,30.31,29.77,30.56,30.19,41237700,1.245272e+09,124576.0,4,0.010333,0.79,30.197407,30.45,29.432,0.285342,0.004019,7.709493e+08,0.029831
1068,2023-11-16,VALE3,73.10,74.10,73.01,74.34,73.75,31235400,2.303803e+09,124576.0,4,0.013680,1.33,73.756158,74.24,71.432,1.358242,0.003112,1.820246e+09,0.037350


In [28]:
df = df.rename(columns={'target': 'tomorrow'})
df

,date,ticker,open,close,min,max,avg,quantity,volume,ibovespa_close,day_of_week,daily_return,price_range,volume_per_quantity,tomorrow,rolling_close_5,rolling_std_5,rolling_return_5,rolling_volume_5,momentum_5
0,2023-01-09,ABEV3,14.29,14.40,14.27,14.47,14.39,24448600,3.519505e+08,109227.0,1,0.007698,0.20,14.395529,14.54,14.244,0.063875,-0.002636,3.625215e+08,0.010952
1,2023-01-09,VALE3,91.53,92.44,91.25,92.99,92.43,19285800,1.782778e+09,109227.0,1,0.009942,1.74,92.439943,93.58,90.256,1.346804,0.007654,2.256946e+09,0.024198
2,2023-01-09,PETR4,23.50,23.87,23.25,24.00,23.77,46385200,1.102685e+09,109227.0,1,0.015745,0.75,23.772347,24.09,23.186,0.631174,0.002385,1.986397e+09,0.029501
3,2023-01-09,BBDC4,14.31,14.56,14.21,14.74,14.53,73674800,1.070745e+09,109227.0,1,0.017470,0.53,14.533399,15.10,14.500,0.463735,-0.000755,1.595722e+09,0.004138
4,2023-01-09,ITUB4,25.00,25.41,24.89,25.66,25.35,25669400,6.508859e+08,109227.0,1,0.016400,0.77,25.356492,25.90,24.510,0.523307,0.004459,7.684781e+08,0.036720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,2023-11-16,ABEV3,13.70,13.67,13.55,13.80,13.66,45772900,6.256602e+08,124576.0,4,-0.002190,0.25,13.668791,13.60,13.486,0.137949,-0.000720,2.831401e+08,0.013644
1066,2023-11-16,PETR4,35.98,35.55,35.50,36.49,35.81,72361200,2.591860e+09,124576.0,4,-0.011951,0.99,35.818366,36.71,35.128,0.801293,0.008634,1.756716e+09,0.012013
1067,2023-11-16,ITUB4,30.00,30.31,29.77,30.56,30.19,41237700,1.245272e+09,124576.0,4,0.010333,0.79,30.197407,30.45,29.432,0.285342,0.004019,7.709493e+08,0.029831
1068,2023-11-16,VALE3,73.10,74.10,73.01,74.34,73.75,31235400,2.303803e+09,124576.0,4,0.013680,1.33,73.756158,74.24,71.432,1.358242,0.003112,1.820246e+09,0.037350


In [29]:
# adding binary target (1 up // 0 down)

df["target"] = (df["tomorrow"] > df["close"]).astype(int)
df

,date,ticker,open,close,min,max,avg,quantity,volume,ibovespa_close,...,daily_return,price_range,volume_per_quantity,tomorrow,rolling_close_5,rolling_std_5,rolling_return_5,rolling_volume_5,momentum_5,target
0,2023-01-09,ABEV3,14.29,14.40,14.27,14.47,14.39,24448600,3.519505e+08,109227.0,...,0.007698,0.20,14.395529,14.54,14.244,0.063875,-0.002636,3.625215e+08,0.010952,1
1,2023-01-09,VALE3,91.53,92.44,91.25,92.99,92.43,19285800,1.782778e+09,109227.0,...,0.009942,1.74,92.439943,93.58,90.256,1.346804,0.007654,2.256946e+09,0.024198,1
2,2023-01-09,PETR4,23.50,23.87,23.25,24.00,23.77,46385200,1.102685e+09,109227.0,...,0.015745,0.75,23.772347,24.09,23.186,0.631174,0.002385,1.986397e+09,0.029501,1
3,2023-01-09,BBDC4,14.31,14.56,14.21,14.74,14.53,73674800,1.070745e+09,109227.0,...,0.017470,0.53,14.533399,15.10,14.500,0.463735,-0.000755,1.595722e+09,0.004138,1
4,2023-01-09,ITUB4,25.00,25.41,24.89,25.66,25.35,25669400,6.508859e+08,109227.0,...,0.016400,0.77,25.356492,25.90,24.510,0.523307,0.004459,7.684781e+08,0.036720,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,2023-11-16,ABEV3,13.70,13.67,13.55,13.80,13.66,45772900,6.256602e+08,124576.0,...,-0.002190,0.25,13.668791,13.60,13.486,0.137949,-0.000720,2.831401e+08,0.013644,0
1066,2023-11-16,PETR4,35.98,35.55,35.50,36.49,35.81,72361200,2.591860e+09,124576.0,...,-0.011951,0.99,35.818366,36.71,35.128,0.801293,0.008634,1.756716e+09,0.012013,1
1067,2023-11-16,ITUB4,30.00,30.31,29.77,30.56,30.19,41237700,1.245272e+09,124576.0,...,0.010333,0.79,30.197407,30.45,29.432,0.285342,0.004019,7.709493e+08,0.029831,1
1068,2023-11-16,VALE3,73.10,74.10,73.01,74.34,73.75,31235400,2.303803e+09,124576.0,...,0.013680,1.33,73.756158,74.24,71.432,1.358242,0.003112,1.820246e+09,0.037350,1


In [30]:


from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)

train = df.iloc[:-100]
test = df.iloc[-100:]

features = [
    'open', 'close', 'min', 'max', 'avg', 'quantity', 'volume',
    'ibovespa_close', 'day_of_week', 'daily_return', 'price_range', 'volume_per_quantity',
    'rolling_close_5', 'rolling_std_5', 'rolling_return_5', 'momentum_5', 'rolling_volume_5'
]

model.fit(train[features], train["target"])



,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,100
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [31]:


from sklearn.metrics import precision_score

preds = model.predict(test[features])
preds = pd.Series(preds, index=test.index)
precision_score(test["target"], preds)



0.4782608695652174

In [32]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [33]:
def backtest(data, model, predictors, start=2500, step=220):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    return pd.concat(all_predictions)

In [34]:
predictions = backtest(df, model, features)

ValueError: No objects to concatenate

In [ ]:
predictions["features"].value_counts()

In [ ]:
precision_score(predictions["target"], predictions["features"])

In [ ]:
predictions["target"].value_counts() / predictions.shape[0]